<a href="https://colab.research.google.com/github/Krishnaugale353/Pnemonia-Detection-using-ML/blob/main/Pnemonia_Detection.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [2]:
#!pip install tensorflow-gpu


In [17]:
from keras.models import Model
from keras.layers import Flatten,Dense
from keras.applications.vgg16 import VGG16
import matplotlib.pyplot as plot
from glob import glob


In [18]:
IMAGESHAPE = [224, 224, 3]
training_data = '/content/drive/MyDrive/chest_xray/train'
testing_data = "/content/drive/MyDrive/chest_xray/test"


In [19]:
!nvidia-smi

Wed Mar  6 05:53:24 2024       
+---------------------------------------------------------------------------------------+
| NVIDIA-SMI 535.104.05             Driver Version: 535.104.05   CUDA Version: 12.2     |
|-----------------------------------------+----------------------+----------------------+
| GPU  Name                 Persistence-M | Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp   Perf          Pwr:Usage/Cap |         Memory-Usage | GPU-Util  Compute M. |
|                                         |                      |               MIG M. |
|=========================================+======================+======================|
|   0  Tesla T4                       Off | 00000000:00:04.0 Off |                    0 |
| N/A   77C    P0              32W /  70W |   6267MiB / 15360MiB |      0%      Default |
|                                         |                      |                  N/A |
+-----------------------------------------+----------------------+--

In [20]:
vgg_model = VGG16(input_shape=IMAGESHAPE, weights='imagenet', include_top=False)


In [21]:
for each_layer in vgg_model.layers:
	each_layer.trainable = False


In [22]:
classes = glob('/content/drive/MyDrive/chest_xray/test/*')


In [23]:
flatten_layer = Flatten()(vgg_model.output)
prediction = Dense(len(classes), activation='softmax')(flatten_layer)


In [24]:
final_model = Model(inputs=vgg_model.input, outputs=prediction)
final_model.summary()


Model: "model_1"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 input_2 (InputLayer)        [(None, 224, 224, 3)]     0         
                                                                 
 block1_conv1 (Conv2D)       (None, 224, 224, 64)      1792      
                                                                 
 block1_conv2 (Conv2D)       (None, 224, 224, 64)      36928     
                                                                 
 block1_pool (MaxPooling2D)  (None, 112, 112, 64)      0         
                                                                 
 block2_conv1 (Conv2D)       (None, 112, 112, 128)     73856     
                                                                 
 block2_conv2 (Conv2D)       (None, 112, 112, 128)     147584    
                                                                 
 block2_pool (MaxPooling2D)  (None, 56, 56, 128)       0   

In [25]:
final_model.compile(
loss='categorical_crossentropy',
optimizer='adam',
metrics=['accuracy']
)


In [26]:
from keras.preprocessing.image import ImageDataGenerator
train_datagen = ImageDataGenerator(rescale = 1./255,
								shear_range = 0.2,
								zoom_range = 0.2,
								horizontal_flip = True)
testing_datagen = ImageDataGenerator(rescale =1. / 255)


In [27]:
training_set = train_datagen.flow_from_directory('/content/drive/MyDrive/chest_xray/train',
												target_size = (224, 224),
												batch_size = 6,
												class_mode = 'categorical')


Found 5218 images belonging to 2 classes.


In [28]:
test_set = testing_datagen.flow_from_directory('/content/drive/MyDrive/chest_xray/test',
											target_size = (224, 224),
											batch_size = 6,
											class_mode = 'categorical')


Found 624 images belonging to 2 classes.


In [29]:
print("Size of the training dataset:", len(training_set))
print("Size of the testing dataset:", len(test_set))

Size of the training dataset: 870
Size of the testing dataset: 104


In [30]:
fitted_model = final_model.fit_generator(
training_set,
validation_data=test_set,
epochs=5,
steps_per_epoch=len(training_set),
validation_steps=len(test_set)
)


Epoch 1/5


<ipython-input-30-0133b02f9370>:1: UserWarning: `Model.fit_generator` is deprecated and will be removed in a future version. Please use `Model.fit`, which supports generators.
  fitted_model = final_model.fit_generator(


870/870 [==============================] - 525s 603ms/step - loss: 0.2369 - accuracy: 0.9222 - val_loss: 0.7725 - val_accuracy: 0.8606
Epoch 2/5
870/870 [==============================] - 154s 177ms/step - loss: 0.1788 - accuracy: 0.9504 - val_loss: 0.8245 - val_accuracy: 0.8814
Epoch 3/5
870/870 [==============================] - 155s 178ms/step - loss: 0.2165 - accuracy: 0.9546 - val_loss: 0.5434 - val_accuracy: 0.9006
Epoch 4/5
870/870 [==============================] - 154s 177ms/step - loss: 0.2088 - accuracy: 0.9578 - val_loss: 0.8593 - val_accuracy: 0.8846
Epoch 5/5
870/870 [==============================] - 154s 178ms/step - loss: 0.1827 - accuracy: 0.9561 - val_loss: 0.4674 - val_accuracy: 0.9199


In [36]:
final_model.save('/content/drive/MyDrive/chest_xray/our_modell.h5')


/usr/local/lib/python3.10/dist-packages/keras/src/engine/training.py:3103: UserWarning: You are saving your model as an HDF5 file via `model.save()`. This file format is considered legacy. We recommend using instead the native Keras format, e.g. `model.save('my_model.keras')`.
  saving_api.save_model(


In [37]:
from keras_preprocessing import image
from keras.models import load_model
from keras.applications.vgg16 import preprocess_input
import numpy as np
model=load_model('/content/drive/MyDrive/chest_xray/our_modell.h5') #Loading our model
img=image.load_img('/content/drive/MyDrive/chest_xray/test/PNEUMONIA/person100_bacteria_475.jpeg',target_size=(224,224))
imagee=image.img_to_array(img) #Converting the X-Ray into pixels
imagee=np.expand_dims(imagee, axis=0)
img_data=preprocess_input(imagee)
prediction=model.predict(img_data)
if prediction[0][0]>prediction[0][1]: #Printing the prediction of model.
	print('Person is safe.')
else:
	print('Person is affected with Pneumonia.')
print(f'Predictions: {prediction}')


1/1 [==============================] - 0s 230ms/step
Person is affected with Pneumonia.
Predictions: [[0. 1.]]


In [38]:
from sklearn.metrics import classification_report, confusion_matrix


# Evaluate the model on the testing set
evaluation = final_model.evaluate(test_set, steps=len(test_set))

# Print overall accuracy
print("Testing Accuracy: {:.2f}%".format(evaluation[1] * 100))

# Make predictions on the test set
predictions = final_model.predict(test_set, steps=len(test_set))

# Convert predictions to binary values (0 or 1)
predicted_classes = np.argmax(predictions, axis=1)

# Get true labels
true_classes = test_set.classes

# Print confusion matrix and classification report
conf_matrix = confusion_matrix(true_classes, predicted_classes)
classification_rep = classification_report(true_classes, predicted_classes, target_names=classes)

print("\nConfusion Matrix:")
print(conf_matrix)

print("\nClassification Report:")
print(classification_rep)


104/104 [==============================] - 9s 84ms/step - loss: 0.4674 - accuracy: 0.9199
Testing Accuracy: 91.99%
104/104 [==============================] - 10s 91ms/step

Confusion Matrix:
[[ 81 153]
 [139 251]]

Classification Report:
                                                  precision    recall  f1-score   support

   /content/drive/MyDrive/chest_xray/test/NORMAL       0.37      0.35      0.36       234
/content/drive/MyDrive/chest_xray/test/PNEUMONIA       0.62      0.64      0.63       390

                                        accuracy                           0.53       624
                                       macro avg       0.49      0.49      0.49       624
                                    weighted avg       0.53      0.53      0.53       624



In [35]:
from sklearn.svm import SVC
from sklearn.pipeline import make_pipeline
from sklearn.preprocessing import StandardScaler
from sklearn.metrics import classification_report, confusion_matrix
import numpy as np

# Assuming you have the training_set and test_set from your CNN code

# Extract images and labels from the training set
train_images, train_labels = training_set.next()

# Reshape the data
train_images_flattened = train_images.reshape(train_images.shape[0], -1)

# Create and train the binary SVM model for normal vs. pneumonia
svm_model = make_pipeline(StandardScaler(), SVC(kernel='linear', C=2))
svm_model.fit(train_images_flattened, np.argmax(train_labels, axis=1))

# Evaluate the model on the testing set
test_images, test_labels = test_set.next()
test_images_flattened = test_images.reshape(test_images.shape[0], -1)

predictions_svm = svm_model.predict(test_images_flattened)

# Print confusion matrix and classification report for SVM
conf_matrix_svm = confusion_matrix(np.argmax(test_labels, axis=1), predictions_svm)
classification_rep_svm = classification_report(np.argmax(test_labels, axis=1), predictions_svm)

print("\nSVM - Confusion Matrix:")
print(conf_matrix_svm)

print("\nSVM - Classification Report:")
print(classification_rep_svm)


SVM - Confusion Matrix:
[[3 0]
 [1 2]]

SVM - Classification Report:
              precision    recall  f1-score   support

           0       0.75      1.00      0.86         3
           1       1.00      0.67      0.80         3

    accuracy                           0.83         6
   macro avg       0.88      0.83      0.83         6
weighted avg       0.88      0.83      0.83         6



In [39]:
from sklearn.svm import SVC
from sklearn.pipeline import make_pipeline
from sklearn.preprocessing import StandardScaler
from sklearn.metrics import classification_report, confusion_matrix
import numpy as np
from sklearn.model_selection import train_test_split

In [ ]:
# Initialize empty lists to store images and labels
train_images_flattened = []
train_labels = []

# Loop through all batches in the training set and extract images and labels
for images_batch, labels_batch in training_set:
    # Flatten and append images
    train_images_flattened.extend(images_batch.reshape(images_batch.shape[0], -1))
    # Append labels
    train_labels.extend(labels_batch)

# Break the loop after iterating through all images
if len(train_images_flattened) == len(training_set.filenames):
    break

In [ ]:
# Split data into training and validation sets
X_train, X_val, y_train, y_val = train_test_split(train_images_flattened, train_labels, test_size=0.2, random_state=42)


In [ ]:
# Create and train the binary SVM model for normal vs. pneumonia
svm_model = make_pipeline(StandardScaler(), SVC(kernel='linear', C=2))
svm_model.fit(X_train, y_train)

In [ ]:
# Evaluate on the validation set
predictions_val = svm_model.predict(X_val)
conf_matrix_val = confusion_matrix(y_val, predictions_val)
classification_rep_val = classification_report(y_val, predictions_val)

print("\nSVM Validation - Confusion Matrix:")
print(conf_matrix_val)

print("\nSVM Validation - Classification Report:")
print(classification_rep_val)

# Optional: Evaluate on the testing set (if provided)
if test_set:
    test_images, test_labels = test_set.images, test_set.labels
    test_images_flattened = test_images.reshape(test_images.shape[0], -1)
    predictions_svm = svm_model.predict(test_images_flattened)

    conf_matrix_svm = confusion_matrix(test_labels, predictions_svm)
    classification_rep_svm = classification_report(test_labels, predictions_svm)

    print("\nSVM - Confusion Matrix:")
    print(conf_matrix_svm)

    print("\nSVM - Classification Report:")
    print(classification_rep_svm)